In [7]:
#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [8]:
data_df = pd.read_csv("../haberman.data",header=None)
data_df.fillna(data_df.median(),inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
data_df.drop_duplicates(inplace=True)

In [9]:
data_df.head()

,Age,Year,Nodes,Survival_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


In [10]:
#s = Data Path p = Number of Cluster col = Label Column Name
def detected_indices_temp(data,p,col,model):
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = 0.4)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)

    # User Optimal Number of Cluster
    km = KMeans(n_clusters = p)
    km.fit(mislabeled.loc[:,mislabeled.columns!=col])
    label = km.labels_
    mislabeled['cluster'] = label

    #OneHotEncoding
    mislabeled = pd.get_dummies(mislabeled,columns=['cluster'])
    #RandomForestClassifier
    x_train,x_test,y_train,y_test = train_test_split(mislabeled.loc[:,mislabeled.columns!=col],mislabeled[col],test_size = 0.38)
    rf = model()
    rf.fit(x_train,y_train)
    predict = rf.predict(mislabeled.loc[:,mislabeled.columns!=col])
    mislabeled['predicted'] = predict
    detected = list()
    cnt = 0
    bad_change = 0
    for i in indices:
        if mislabeled[col][i] != mislabeled['predicted'][i]:
            detected.append(i)
    for i in detected:
        if i in noisy_indices:
            cnt+=1
        else:
            bad_change+=1
    return pd.DataFrame(data={'Total Noise':[len(noisy_indices)], 'Total Detected labels':[len(detected)], 'True Identified labels':[cnt], 'False identified labels':[bad_change], 'Total Noise Ratio':len(noisy_indices)/data.shape[0], 'True Identified Ratio':cnt/len(noisy_indices)})

    print(len(noisy_indices),len(detected),cnt,bad_change)        

In [11]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = detected_indices_temp(data_df,10,'Survival_status',model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
result

,False identified labels,Total Detected labels,Total Noise,Total Noise Ratio,True Identified Ratio,True Identified labels,Model
0,30,87,116,0.401384,0.491379,57,SVC
1,46,63,116,0.401384,0.146552,17,Random Forest
2,85,124,116,0.401384,0.336207,39,Naive Bayes
3,62,135,116,0.401384,0.629310,73,Logistic Regression
4,36,61,116,0.401384,0.215517,25,Decision Tree


In [12]:
# #k=10
# %matplotlib qt  
# def plot_Elbow(features):
#     sse = {}
#     for no_of_cluster in range(2,50):
#         kmeans = KMeans(n_clusters=no_of_cluster).fit(features)
#         sse[no_of_cluster] = kmeans.inertia_
#     plt.figure()
#     plt.plot(list(sse.keys()), list(sse.values()))
#     plt.xlabel("Number of cluster")
#     plt.ylabel("SSE")
#     plt.show()

In [13]:
plot_Elbow(data_df.loc[:,data_df.columns!='Survival_status'])

NameError: name 'plot_Elbow' is not defined

In [ ]:
data_df.shape